In [3]:
import requests
import rsa
import json
import binascii

def hex_string_to_bytes(hex_string):
    return binascii.unhexlify(hex_string)

def bytes_to_hex_string(byte_array):
    return binascii.hexlify(byte_array).decode()

def login(user_name, password):
    url = "http://127.0.0.1/PortfolioDEV/Services/Framework/AuthenticationService.svc"
    session_url = url + "/session"
    login_url = url + "/login"

    response = requests.get(session_url)
    if response.status_code != 200:
        raise Exception('GET /session {}'.format(response.status_code))
    session = json.loads(response.text)
    m = hex_string_to_bytes(session['rsa']['M'])
    e = hex_string_to_bytes(session['rsa']['E'])
    rsa_key = rsa.PublicKey(int.from_bytes(m, byteorder='big'), int.from_bytes(e, byteorder='big'))
    session_plus_password = session['id'] + "**" + password
    encrypted_password = rsa.encrypt(session_plus_password.encode(), rsa_key)
    headers = {'content-Type': 'application/json'}
    data = {
        'sessionId': session['id'],
        'username': user_name,
        'password': bytes_to_hex_string(encrypted_password)
    }
    response = requests.post(login_url, headers=headers, data=json.dumps(data))
    if response.status_code != 200:
        raise Exception('POST /login {}'.format(response.status_code))
    auth_cookie = response.headers['Set-Cookie'].split(',')[1]
    return session['id'], auth_cookie


ModuleNotFoundError: No module named 'rsa'

In [8]:
session_id, auth_cookie = login('admin', 'admin')
print('Session ID:', session_id)
print('Auth Cookie:', auth_cookie)



Session ID: f70258e533bc47c48fb8ed180b94d55a
Auth Cookie:  04-Nov-2023 07:41:45 GMT; path=/; HttpOnly
